This notebook tries to overcome the difficulties we experienced when trying to load more than 25k instances.

In [8]:
import pandas as pd

df = pd.read_csv("transfer_10.csv").dropna()
columns_to_keep = ["hashname", "human_transcript", "unnormalized_transcript", "duration"]
c1 = ~df.was_in_samples
c2 = df.sim >= 0.8
df = df.loc[c1 & c2, columns_to_keep]
print(df.duration.sum()/3600)
df = df.rename(columns={"human_transcript":"sentence"})
df.shape

199.47182222222224


(44446, 4)

In [2]:
df.columns

Index(['hashname', 'sentence', 'unnormalized_transcript'], dtype='object')

In [3]:
test_df = df.iloc[0:500, :].copy()
train_df = df.iloc[500:, :].copy()


train_df["audio"] = train_df.hashname.apply(lambda s: "transfer_10/"+s)
test_df["audio"] = test_df.hashname.apply(lambda s: "transfer_10/"+s)


import datasets
from datasets import load_dataset, load_metric, Audio

train_dataset = datasets.Dataset.from_pandas(train_df)
test_dataset = datasets.Dataset.from_pandas(test_df)

train_dataset = train_dataset.cast_column("audio", Audio())
test_dataset = test_dataset.cast_column("audio", Audio())

In [4]:
import os
# Use old or new vocab?
os.system("cp vocab_300.json vocab.json")


from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
    "./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token=" ")

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)

import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:

from transformers import Trainer
from transformers import TrainingArguments
from transformers import Wav2Vec2ForCTC
from typing import Any, Dict, List, Optional, Union
from dataclasses import dataclass, field
import torch


def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(
        audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch


train_mapped = train_dataset.map(
    prepare_dataset, remove_columns=train_dataset.column_names)
test_mapped = test_dataset.map(
    prepare_dataset, remove_columns=test_dataset.column_names)
print("Data Preparation Complete!")

  0%|          | 0/43946 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

Data Preparation Complete!


In [11]:
modelname = "25_normalised_200h"
repo_name = modelname
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")
def compute_metrics(pred):
    import numpy as np
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}

max_input_length_in_sec = 20
train_mapped = train_mapped.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

model.freeze_feature_extractor()

from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=4,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  num_train_epochs=4,
  gradient_checkpointing=True,
  fp16=True,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=5,
  push_to_hub=False,
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_mapped,
    eval_dataset=test_mapped,
    tokenizer=processor.feature_extractor,
)

trainer.train()

  0%|          | 0/44 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/facebook/wav2vec2-xls-r-300m/resolve/main/config.json from cache at /home/peterr/.cache/huggingface/transformers/dabc27df63e37bd2a7a221c7774e35f36a280fbdf917cf54cadfc7df8c786f6f.a3e4c3c967d9985881e0ae550a5f6f668f897db5ab2e0802f9b97973b15970e6
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,

Epoch,Training Loss,Validation Loss,Wer,Cer
1,3.044100,0.188277,0.173580,0.049340
2,0.184200,0.123732,0.099893,0.031663
3,0.128000,0.111795,0.085531,0.027965
4,0.101500,0.107521,0.077653,0.026187


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to 25_normalised_200h/checkpoint-680
Configuration saved in 25_normalised_200h/checkpoint-680/config.json
Model weights saved in 25_normalised_200h/checkpoint-680/pytorch_model.bin
Feature extractor saved in 25_normalised_200h/checkpoint-680/preprocessor_config.json
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to 25_normalised_200h/checkpoint-1360
Configuration saved

TrainOutput(global_step=2720, training_loss=0.5806092739105224, metrics={'train_runtime': 33744.6133, 'train_samples_per_second': 5.157, 'train_steps_per_second': 0.081, 'total_flos': 8.518012037007127e+19, 'train_loss': 0.5806092739105224, 'epoch': 4.0})